# 1. Proof of Work

## 1.1 参照記事
* https://qiita.com/haystacker/items/a7b0c2552e1fd937a789

## 1.2ブロック

### 1.2.1 ブロックとは？
* ブロックチェーンの分散型データベースに格納されるデータの1まとまり。

### 1.2.2 ブロックの構造
* ヘッダ
    * ブロックの情報を識別するためのデータ
* トラザクション
    * トランザクションの詳細が含まれている
* ブロックハッシュ値
    * ブロックの一種の識別子であり、ブロックのデータが改ざんされていないことを確認するために使用

### 1.2.2 ヘッダの構造
* インデックス
    * ブロックのインデックス番号
* タイムスタンプ
    * ブロックが生成された日時
* ナンス
    * ブロックハッシュの生成に使用されるランダムな値
* 前のブロックのハッシュ値
    * 前のブロックの一意の識別子であり、ブロックチェーン内の前のブロックに対する参照
* マークルルート
    * ブロック内のすべてのトランザクションのハッシュ値をマージすることで生成される

In [2]:
# ブロックヘッダーのイニシャライザ
class Block:
    def __init__(self, index, timestamp, nonce, previous_block_hash, merkle_root, transactions):
        self.index = index
        self.timestamp = timestamp
        self.nonce = nonce
        self.previous_block_hash = previous_block_hash
        self.merkle_root = merkle_root

## 1.3 トランザクション

### 1.3.1 トランザクションとは？
* ブロックチェーン上で行われる取引のこと
* 保持するデータは用途によって変わる
* トランザクションが利用される流れ
    1. トランザクションプールに新規登録される
    2. マイナー（採掘者）によってトランザクションプールから取り出される
    3. 取り出されたトランザクションが新規ブロックに挿入される
    4. ブロックがブロックチェーンに登録される
    5. トランザクションが承認状態になる

In [3]:
from queue import Queue

class Blockchain:
    def __init__(self):
        self.transaction_pool = Queue() # トランザクションプール
        
    # トランザクションを追加する
    ## トランザクションをtransaction_poolにPUTする
    ## Queue.putはデータの挿入
    ### https://docs.python.org/3/library/queue.html#queue.Queue.put
    def add_transaction(self, transaction):
        self.transaction_pool.put(transaction)
        
    # マイニングする
    ## transaction_poolからトランザクションを取り出し、リストとしてセットする
    ## その後ブロックチェーンとしてブロックを追加する際に transaction_poolを初期化する
    def mine_block(self):
        ## ...
        new_block_transaction = list(self.transaction_pool.queue)
        ## ...
        while True:
            ## ...
            # ハッシュ値が要求された難易度より小さい場合、ブロックを追加して終了
            if new_block_hash[:self.difficulty] == '0' * self.difficulty:
                self.transaction_pool = Queue()
                ## ...

## 1.4 ハッシュとマークルルート
### 1.4.1 ハッシュとは？
* ハッシュとは、ハッシュ関数でデータから固定長の一意の値を生成したもの
    * ハッシュ関数
        * 入力した値に対して、特定のルールで全く異なる値を返す関数
        * https://wa3.i-3-i.info/word11948.html

In [2]:
import hashlib

# string（入力値）をSHA-256（ハッシュ関数）で変換する
## SHA-256
### https://e-words.jp/w/SHA-256.html
### cf: 文字列の前のbについて
#### https://www.tohoho-web.com/python/types.html
string1 = b"Hello World!"
hashlib.sha256(string1).hexdigest()

'7f83b1657ff1fc53b92dc18148a1d65dfc2d4b1fa3d677284addd200126d9069'

In [4]:
# 同じ入力値に同じハッシュ関数を用いた場合、同じ値が出力される
string2 = b"Hello World!"
hashlib.sha256(string2).hexdigest()

'7f83b1657ff1fc53b92dc18148a1d65dfc2d4b1fa3d677284addd200126d9069'

In [6]:
# 入力値が異なれば同じハッシュ関数からの出力値も異なる
string3 = b"Hello World"
hashlib.sha256(string3).hexdigest()

'a591a6d40bf420404a011733cfb7b190d62c65bf0bcda32b57b277d9ad9f146e'

##### **※ブロックチェーン上の全てのブロックやトランザクションにはハッシュ値が割り当てられているため、すべてのデータが一意に識別され、改ざんされていないことが確認される**

## 1.4.2 マークルルートとは？
* ブロックチェーンにおいて複数のトランザクションのハッシュ値を1つのハッシュ値にまとめるために使用されるデータ構造
    * マークルルートはハッシュ値を木構造（ツリー状）に並べたものであり、ルートのハッシュ値をブロックのヘッダに保存すること。それにより複数のトランザクションの一意性を確認する
* マークルルートは複数のトランザクションが含まれるブロックに対して使用される。
    * トランザクションのハッシュ値がリーフノードに配置され、隣接するリーフノードのハッシュ値が親ノードのハッシュ値として計算される。これを繰り返すことで全体のハッシュ値がルートに設定される
        * リーフノードとは木の様に枝分かれするデータ構造である木構造（ツリー）を構成する要素のうち末端の要素のこと
            * https://wa3.i-3-i.info/word14931.html
* ブロック内のトランザクションが変更された場合そのハッシュ値も変更され、さらにその親のノードのハッシュ値も変更されるため、ルートのハッシュも変更されることになり、改ざんを検出することができる

In [7]:
# SHA-256でハッシュ計算する関数
def calculate_hash(self):
    block_string = json.dumps({
        'index': self.index,                              # インデックス
        'timestamp': self.timestamp,                      # タイムスタンプ
        'nonce': self.nonce,                              # ナンス
        'previous_block_hash': self.previous_block_hashe, # 前のブロックのハッシュ値
        'merkle_root': self.merkle_root,                  # マークルルート
    }, sort_keys=True).encode()
    return hashlib.sha256(block_string).hexdigest()

* 前のブロックのハッシュ値は、マイニングする際に前のチェーンの`block_hash`から取得する

In [8]:
# マイニングする関数
def mine_block(self):
    # 直前のブロックを取得する
    previous_block = self.chain[-1]
    # ...
    new_block_previous_hash = previous_block['block_hash']

### 1.4.3 マークルルートの実装
* マークルルートは以下のアルゴリズムで処理する
    1. トランザクション数が0の場合は空文字のハッシュ値を返す
    2. トランザクション数が1の場合はそのトランザクションのハッシュ値を返す
    3. トランザクション数が奇数の場合は最後の要素を複製して偶数にする
    4. 2つのトランザクションを組み合わせてハッシュ値を計算しリストに格納する
    5. 「3」~「4」を再帰的に繰り返す

In [9]:
# マークルルートを計算する関数
def calculate_merkle_root(self, transactions):
    # 1.トランザクション数が0の場合は空文字のハッシュ値を返す
    if len(transactions) == 0:
        return hashlib.sha256(b'').hexdigest()
    # 2.トランザクション数が1の場合はそのトランザクションのハッシュ値を返す
    if len(transactions) == 1:
        return hashlib.sha256(transactions[0].encode()).hexdigest()
    # 3.トランザクション数が奇数の場合は最後の要素を複製して偶数にする
    if len(transactions) %2 == 1:
        transactions.append(transactions[-1])
    # ハッシュ値を格納するリストを初期化
    hashes = []
    # 4.2つのトランザクションを組み合わせてハッシュ値を計算しリストに格納する
    for i in range(0, len(transactions), 2):
        transaction_pair = transactions[i] + transactions[i+1]
        hash_value = hashlib.sha256(transaction_pair.encode()).hexdigest()
        hashes.append(hash_value)
    # 再帰的にマークルルートを計算する
    return self.calculate_merkle_root(hashes)